In [1]:
# on server: 'screen' ,then start script
# use 'strg+a d' to return to terminal
# use 'screen -r' to return to screen
import numpy as np
import json
import os
import math

# from keras.utils import plot_model
from keras.models import model_from_json
os.environ['CUDA_VISIBLE_DEVICES']='2'

import preprocess_data as ppd
import make_RNN_models as mrm
import train_slices as ts

from tqdm import tqdm

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Using TensorFlow backend.


In [2]:
# https://towardsdatascience.com/nlp-sequence-to-sequence-networks-part-1-processing-text-data-d141a5643b72
path='testpath/'
# rnn parameters
hidden_size = 100 #100 is the standard
batch_size = 100 #for the training on the GPU this to be has to very large, otherwise the GPU is used very inefficiently
epochs = 1#50

size=10000

#glove embedding parameters
glove_dir = '../glove/glove.6B.100d.txt'
embedding_dim = 100

In [3]:
#open SQuAD-dataset and extract the relevant data from the json-file
#to a easier readable/accessible dictionary
with open('SQuAD/train-v2.0.json') as file:
    train=json.load(file)
train_qid=[]
train_context=[]
train_question=[]
train_answer=[]
train_new={'context':train_context,'question':train_question,'answer':train_answer,'qid':train_qid}
for j,data in enumerate(train['data']):
    for i,paragraph in enumerate(data['paragraphs']):
        context=paragraph['context']
        for qas in paragraph['qas']:
            
            #create a dataset with only the answerable questions
            #add a bos and eos token to the target
            if (qas['is_impossible']==False):
                c=qas['answers'][0]['text'].lower()
                
                train_new['qid'].append(qas['id'])
                train_new['context'].append(context.lower())
                train_new['question'].append(qas['question'].lower())
                train_new['answer'].append('START_ '+c+' _END')
            else:
                train_new['qid'].append(qas['id'])
                train_new['context'].append(context.lower())
                train_new['question'].append(qas['question'].lower())
                train_new['answer'].append('START_ '+str(qas['answers'])+' _END')
print(len(train_new['qid']))
print(len(train_new['context']))
print(len(train_new['question']))
print(len(train_new['answer']))

130319
130319
130319
130319


In [4]:
data_info=ppd.get_data_info([train_new['context'],
                             train_new['question'],
                             train_new['answer']])

In [5]:
# #FIX_ME: add glove download
# # https://nlp.stanford.edu/projects/glove/
# #get glove embeddings
# # https://medium.com/@sabber/classifying-yelp-review-comments-using-cnn-lstm-and-pre-trained-glove-word-embeddings-part-3-53fcea9a17fa
# print('getting the glove embeddings')
# embeddings_index = {}
# f = open(glove_dir)
# for line in tqdm(f):
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Found %s word vectors.' % len(embeddings_index))

In [6]:
# #extract the glove-embedding to a matrix
# context_embedding_matrix = np.zeros((data_info['len_context_vocab'], embedding_dim))
# for word, i in data_info['context_token_to_int'].items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         context_embedding_matrix[i] = embedding_vector

# question_embedding_matrix = np.zeros((data_info['len_question_vocab'], embedding_dim))
# for word, i in data_info['question_token_to_int'].items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         question_embedding_matrix[i] = embedding_vector

# answer_embedding_matrix = np.zeros((data_info['len_answer_vocab'], embedding_dim))
# for word, i in data_info['answer_token_to_int'].items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         answer_embedding_matrix[i] = embedding_vector

In [7]:
# for slice_size in range(12,math.ceil(len(train_new['context'])/size)):
#     ts.train_slices([train_new['context'],train_new['question'],train_new['answer']],
#                     data_info,
#                     [context_embedding_matrix,question_embedding_matrix,answer_embedding_matrix],
#                     hidden_size,
#                     embedding_dim,
#                     batch_size,
#                     epochs,
#                     slice_size,
#                     size,
#                     path)

In [20]:
path='models/'
print('start inference')
with open(path+'encoder_model.json', 'r') as encoder_json_file:
    loaded_model_json = encoder_json_file.read()
    encoder_model = model_from_json(loaded_model_json)
encoder_model.load_weights(path+'encoder_model.h5')
encoder_json_file.close()
    
with open(path+'decoder_model.json', 'r') as decoder_json_file:
    loaded_model_json = decoder_json_file.read()
    decoder_model = model_from_json(loaded_model_json)
decoder_model.load_weights(path+'decoder_model.h5')
decoder_json_file.close()
########################################################
qid_to_answer_dict={}
for slice_size in range(math.ceil(len(train_new['context'])/size)):
    print('inference on part %s of the dataset' % slice_size)
    input_data=ppd.process_data([train_new['context'][size*slice_size:size*(slice_size+1)],
                                 train_new['question'][size*slice_size:size*(slice_size+1)],
                                 train_new['answer'][size*slice_size:size*(slice_size+1)]]
                                ,data_info)
    #####################################################
    print('len_ans_dict',len(qid_to_answer_dict))
    for seq_index in tqdm(range(len(train_new['context'][size*slice_size:size*(slice_size+1)]))):
#         decoded_sentence = ppd.decode_sequence(input_data['encoder_input']['context_encoder_input'][seq_index:seq_index+1],
#                                                 input_data['encoder_input']['question_encoder_input'][seq_index:seq_index+1],
#                                                 data_info['answer_token_to_int'],
#                                                 data_info['answer_int_to_token'],
#                                                 encoder_model,
#                                                 decoder_model)
        qid_to_answer_dict[train_new['qid'][seq_index+(slice_size*size)]]=1#decoded_sentence
print('len_ans_dict',len(qid_to_answer_dict))
# with open(path+'answers.json', 'w') as file:
#     json.dump(qid_to_answer_dict, file)
# file.close()

start inference
inference on part 0 of the dataset


100%|██████████| 10000/10000 [00:00<00:00, 1384076.03it/s]

len_ans_dict 0
inference on part 1 of the dataset



100%|██████████| 10000/10000 [00:00<00:00, 1262888.11it/s]

len_ans_dict 10000
inference on part 2 of the dataset


...

len_ans_dict 100000
inference on part 11 of the dataset



100%|██████████| 10000/10000 [00:00<00:00, 1186641.77it/s]

len_ans_dict 110000
inference on part 12 of the dataset



100%|██████████| 319/319 [00:00<00:00, 563598.56it/s]

len_ans_dict 120000
inference on part 13 of the dataset
len_ans_dict 130000
len_ans_dict 130319


In [9]:
# with open('ddas.txt','a') as file:
#     file.write('f')

In [10]:
# with open('SQuAD/answers.json', 'w') as file:
#     json.dump(qid_to_answer_dict, file)

In [11]:
# with open('SQuAD/answers.json') as file:
#     answers_json=json.load(file)

In [12]:
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [13]:
# plot_model(model, to_file=path+'/model.png')

In [14]:
# print('save model')
# if not os.path.isdir(path):
#     os.makedirs(path)
# model.save_weights(path+str('baseline_model.h5')) #save weights
# model_json = model.to_json()
# with open(path+str('baseline_model.json'),'w') as json_file:
#     json_file.write(model_json)